<a href="https://colab.research.google.com/github/jeffersonramelo/Peper-relato-integrado-DIF-DIF/blob/main/Painel_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas openpyxl

In [10]:
import pandas as pd

# Caminho para o seu arquivo XLSX
arquivo_xlsx = '/content/merged_data_final todas.xlsx'

# Lendo o arquivo XLSX
try:
    df_panel = pd.read_excel(arquivo_xlsx, engine='openpyxl')
    # Exibindo as primeiras linhas do DataFrame para verificar os df
    print(df.head())
except Exception as e:
    print(f"Erro ao ler o arquivo: {e}")

   id codigo                                Nome_Companhia        Nome  \
0   1   AESB                        AES Brasil Energia S/A  AES Brasil   
1   2   AMBP  AMBIPAR PARTICIPAÇÕES E EMPREENDIMENTOS S.A.     Ambipar   
2   3   AZUL                                     AZUL S.A.        Azul   
3   4   CEAB                                C&A MODAS S.A.   Cea Modas   
4   5   SBSP        CIA SANEAMENTO BÁSICO ESTADO SÃO PAULO      Sabesp   

    SetorEconômicob3    SubsetorBovespa   SetorEconomatica  \
0  Utilidade pública   Energia elétrica   Energia Elétrica   
1  Utilidade pública  Água e saneamento             Outros   
2   Bens industriais         Transporte  Transporte Serviç   
3    Consumo cíclico           Comércio           Comércio   
4  Utilidade pública  Água e saneamento             Outros   

   ValorMercado30Abr11  ValorMercado30Mai11  ValorMercado30Jun11  ...  \
0                  NaN                  NaN                  NaN  ...   
1                  NaN              

In [11]:
from scipy.stats.mstats import winsorize

# Assumindo que df já foi definido e filtrado para os anos de 2020, 2021 e 2022
for col in ['PatrimLiqDez2010', 'LucroLiquidoDez2010']:
    # Aplicando winsorize em cada coluna selecionada com limites de 1% para os outliers em ambos os lados da distribuição
    df_panel[col] = winsorize(df_panel[col], limits=[0.01, 0.01])


In [7]:
pip install linearmodels

In [15]:
# Supondo que df_panel seja seu DataFrame original

# Configurando 'id' e 'ano' como índices MultiIndex se ainda não estiver configurado
if not isinstance(df_panel.index, pd.MultiIndex):
    df_panel = df_panel.set_index(['id', 'ano'])

# Verificar se a configuração foi realizada corretamente
print(df_panel.index)

# Agora, preparar novamente 'y' e 'X' com o DataFrame corrigido
y = df_panel['PU30Jun11']
X = df_panel[['PatrimLiqDez2010', 'LucroLiquidoDez2010', 'd_divulga']]

# Não esqueça de adicionar a constante a 'X' se estiver usando statsmodels junto com linearmodels
X = sm.add_constant(X)

# Ajustando os modelos como antes
modelo_ef = PanelOLS(y, X, entity_effects=True).fit()
modelo_ea = RandomEffects(y, X).fit()
modelo_pooled = PooledOLS(y, X).fit()

# Comparando os modelos
resultado_comparacao = compare({"Efeitos Fixos": modelo_ef, "Efeitos Aleatórios": modelo_ea, "Pooled OLS": modelo_pooled})
print(resultado_comparacao)


MultiIndex([(  1, 2010),
            (  2, 2010),
            (  3, 2010),
            (  4, 2010),
            (  5, 2010),
            (  6, 2010),
            (  7, 2010),
            (  8, 2010),
            (  9, 2010),
            ( 10, 2010),
            ...
            (383, 2022),
            (384, 2022),
            (385, 2022),
            (385, 2022),
            (385, 2022),
            (386, 2022),
            (387, 2022),
            (388, 2022),
            (389, 2022),
            (407, 2022)],
           names=['id', 'ano'], length=5264)


/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:2618: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:882: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


                            Model Comparison                            
                         Efeitos Fixos Efeitos Aleatórios     Pooled OLS
------------------------------------------------------------------------
Dep. Variable                PU30Jun11          PU30Jun11      PU30Jun11
Estimator                     PanelOLS      RandomEffects      PooledOLS
No. Observations                  3241               3241           3241
Cov. Est.                   Unadjusted         Unadjusted     Unadjusted
R-squared                       0.0010             0.0002         0.0002
R-Squared (Within)              0.0010          4.829e-05      9.073e-06
R-Squared (Between)            -0.0839             0.0006         0.0006
R-Squared (Overall)            -0.0180             0.0002         0.0002
F-statistic                     0.9462             0.1865         0.2582
P-value (F-stat)                0.4173             0.9056         0.8555
=====================     ============    =========